# Selection des gares à étudier

__Auteur :__ 

Steve Caron

__Présentation :__ 

Ce script permet de trier les gares de l'API SNCF en fonction de leur fréquentation sur l'année 2022.

Il retourne les données des gares dans un fichier json

__Inputs :__ 

Un fichier data/gare_frequentation_inclues.json contenant les informations sur les gares (Créé à partir de ajout_reseau.ipynb)

__Params :__ 

nb_valeurs : nombre de gares que l'on veut classer

In [1]:
import pandas as pd

In [2]:
nb_valeurs = 200

In [3]:
def lire_csv(nom_fichier):
    '''Cette fonction permet de lire un fichier csv et de l'ouvrir dans un dataframe'''
    dataframe = pd.read_csv(nom_fichier)
    return dataframe

In [5]:
def fichier_to_dataframe(chemin: str):
    '''Cette fonction ouvre un fichier json ou csv et place son contenu dans un dataframe'''

    #J'utilise split pour connaitre l'extension de mon fichier
    extension = chemin.split(".")[-1]
    if extension == "csv":
        dataframe = pd.read_csv(chemin)
    elif extension == "json":
        dataframe = pd.read_json(chemin)
    else:
        print("L'extension du fichier doit être csv ou json")

    return dataframe

In [6]:
def clean_data(dataframe):
    '''Cette fonction selectionne deux colonne sur un dataframe, cast la colonne code en int, et ajoute un prefixe'''
    
    dataframe_trois_colonnes = dataframe[["code_uic","total_voyageurs_2022","networks"]]
    # Je cast la colonne code_uic en string
    dataframe_trois_colonnes.loc[:,"code_uic"] = dataframe_trois_colonnes.loc[:,"code_uic"].astype("string")
    # J'applique une fonction lambda sur la colone code_uic pour ajouter un prefix à toutes les valeurs
    dataframe_trois_colonnes.loc[:,"code_uic"] = dataframe_trois_colonnes.loc[:,"code_uic"].apply(lambda x: "stop_area:SNCF:"+x)
    # Je renomme la colonne
    dataframe_trois_colonnes = dataframe_trois_colonnes.rename(columns={'code_uic':'id'})

    
    return dataframe_trois_colonnes

In [7]:
def joindre_dataframe(df1,df2):
    '''Cette fonction permet de joindre deux dataframe'''
    df3 = pd.merge(df1,df2, how='inner', on="id")
    return df3

In [8]:
def topn(df,nb_valeurs):
    '''Cette fonction classe un dataframe selon la frequentation des gare renvoie les n premières valeurs'''

    #Range les enregistrements en fonction du nombre de voyageur
    df_ranked = df.sort_values("total_voyageurs_2022", ascending=False)
    #Les 100 premières valeur
    df_n = df_ranked.iloc[:nb_valeurs]

    return df_n

In [9]:
def main(nb_valeurs):
    #Creation du dataframe
    dataframe_brutes = fichier_to_dataframe("data/gare_frequentation_inclues.json")
    #Nettoyage des données
    data_frequentation = clean_data(dataframe_brutes)
    #Recupération des information des gares
    data_gare = fichier_to_dataframe("data/liste_gare.csv")
    #Jointure des deux tables
    tables_jointes = joindre_dataframe(data_frequentation,data_gare)
    # Classement des n premieres valeurs
    top_gares = topn(tables_jointes,nb_valeurs)
    # Enregistrement en json
    top_gares.to_json(f"data/top{nb_valeurs}gare.json")

In [ ]:
main(nb_valeurs)